In [ ]:
#|default_exp optimizer.utils
#|default_cls_lvl 2

# Optimizer Utils
> bUtilities for fastxtend optimizers

In [ ]:
#|export
from __future__ import annotations

from fastcore.basics import even_mults

from fastxtend.imports import *

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|exporti
def _convert_params(o:list, **defaults) -> list:
    "Convert fastai param_lists to PyTorch param_groups, adding defaults if group doesn't have it"
    splitter = []
    for group in o:
        if isinstance(group, dict):
            splitter.append({**defaults, **group})
        else:
            splitter.append({'params':group, **defaults})
    return splitter

In [ ]:
#|exporti
class FastaiOptimizerAdapter:
    "Base for adding fastai optimizer functionality to PyTorch optimizers"
    _keep_on_clear = ['force_train', 'do_wd']

    def get_params(self,
        n:slice|int=slice(None), # Extended slicing over the optimizer `param_lists`
        with_grad:bool=False # Get all param tuples. If `True` select only those with a gradient
    ):
        "Slice of parameters and parameter states"
        return L((p, self.state[p]) for pg in self.param_groups[n] for p in pg['params']
                    if (hasattr(p, 'grad') and p.grad is not None) or with_grad==False)

    def clear_state(self):
        "Reset the state of the optimizer"
        for p,state in self.get_params():
            self.state[p] = {k: state[k] for k in self._keep_on_clear if k in state}

    def _set_require_grad(self,
        rg:bool, # Requires grad: if `True` sets gradient for parameters, else uses state `state["force_train"]`
        p:Tensor, # Parameter to set gradient
        state:dict, # Parameter's state dict
    ):
        p.requires_grad_(rg or state.get('force_train', False))

    def freeze_to(self, n:int):
        "Freeze parameter groups up to `n`"
        self.frozen_idx = n if n >= 0 else len(self.param_groups) + n
        if self.frozen_idx >= len(self.param_groups):
            warn(f"Freezing {self.frozen_idx} groups; model has {len(self.param_groups)}; whole model is frozen.")
        for o in self.get_params(slice(n, None)):
            self._set_require_grad(True, *o)
        for o in self.get_params(slice(None, n)):
            self._set_require_grad(False, *o)

    def freeze(self):
        "Freeze up to last parameter group"
        assert(len(self.param_groups) > 1)
        self.freeze_to(-1)

    def unfreeze(self):
        "Unfreeze the entire model"
        self.freeze_to(0)

    @property
    def hypers(self):
        return [{k:v for k,v in pg.items() if k != 'params'} for pg in self.param_groups]

    def set_hypers(self, **kwargs):
        "`set_hyper` for all `kwargs`"
        L(kwargs.items()).starmap(self.set_hyper)

    def _set_hyper(self, k, v):
        "Set the value(s) in `v` for hyper-parameter `k`"
        for v_,h in zip(v, self.param_groups):
            h[k] = v_

    def set_hyper(self, k, v):
        "Set the value(s) in `v` for hyper-parameter `k`"
        if isinstance(v, slice):
            if v.start:
                v = even_mults(v.start, v.stop, len(self.param_groups))
            else:
                v = [v.stop/10]*(len(self.param_groups)-1) + [v.stop]
        v = L(v, use_list=None)
        if len(v)==1:
            v = v*len(self.param_groups)
        assert len(v) == len(self.param_groups), f"Trying to set {len(v)} values for {k} but there are {len(self.param_groups)} parameter groups."
        self._set_hyper(k, v)

    @property
    def param_lists(self):
        return [pg['params'] for pg in self.param_groups]

    @param_lists.setter
    def param_lists(self, v):
        for pg,v_ in zip(self.param_groups,v):
            pg['params'] = v_

    def convert_params(self, params, **defaults):
        "Convert fastai param_lists to PyTorch param_groups, adding defaults if group doesn't have it"
        params = L(params)
        return _convert_params(params, **defaults) if isinstance(params[0], (L,list)) else params

In [ ]:
show_doc(FastaiOptimizerAdapter)